In [ ]:
import torch  # PyTorch kütüphanesini içe aktarır.
from torch.utils.data import DataLoader, Subset, Dataset  # Veri yükleme ve işleme için gerekli sınıfları içe aktarır.
from torchvision.datasets import EMNIST  # EMNIST veri kümesini içe aktarır.
from torchvision import transforms  # Görüntü dönüşümleri için gerekli sınıfları içe aktarır.
from diffusers import UNet2DModel, DDPMScheduler  # Diffüzyon modelleri için gerekli sınıfları içe aktarır.
from torch.optim import AdamW  # AdamW optimizasyon algoritmasını içe aktarır.
from accelerate import Accelerator  # Hızlandırıcıyı içe aktarır (GPU/TPU kullanımı için).
from PIL import Image  # Görüntü işleme için Pillow kütüphanesini içe aktarır.
from torchmetrics.image.ssim import StructuralSimilarityIndexMeasure  # SSIM (Yapısal Benzerlik Ölçüsü) metriğini içe aktarır.
from torchmetrics.image.fid import FrechetInceptionDistance  # FID (Fréchet Inception Distance) metriğini içe aktarır.
from torchmetrics.image.inception import InceptionScore  # Inception Score metriğini içe aktarır.
import torch.nn.functional as F  # Fonksiyonel sinir ağı araçlarını içe aktarır.
from tqdm.auto import tqdm  # Döngüler için ilerleme çubuğu gösterimini içe aktarır.
import os  # İşletim sistemi etkileşimleri için os modülünü içe aktarır.
import json  # JSON verisiyle çalışmak için json modülünü içe aktarır.
import matplotlib.pyplot as plt  # Grafik çizmek için matplotlib kütüphanesini içe aktarır.
import pandas as pd  # Veri analizi için pandas kütüphanesini içe aktarır.
from torch.optim.lr_scheduler import ReduceLROnPlateau  # Öğrenme oranını ayarlamak için ReduceLROnPlateau'yu içe aktarır.
# === 1. Veri Kümesi Kurulumu ===

class RemapLabels(Dataset):  # Etiketleri yeniden eşlemek için özel bir Dataset sınıfı tanımlar.
    def __init__(self, subset, label_map):  # Sınıfın kurucu metodu.
        self.subset = subset  # Alt küme verisini saklar.
        self.label_map = label_map  # Etiket eşleme sözlüğünü saklar.

    def __getitem__(self, idx):  # Veri kümesinden bir öğe almak için gerekli olan getitem metodunu tanımlar.
        x, y = self.subset[idx]  # Alt kümeden veriyi alır.
        return x, self.label_map[y]  # Yeniden eşlenmiş etiketle birlikte veriyi döndürür.

    def __len__(self):  # Veri kümesinin uzunluğunu döndüren len metodunu tanımlar.
        return len(self.subset)  # Alt kümenin uzunluğunu döndürür.


def get_letter_emnist_dataset():  # Harf EMNIST veri kümesini almak için bir fonksiyon tanımlar.
    full_dataset = EMNIST(  # Tam EMNIST veri kümesini yükler.
        "./data",  # Veri kümesinin saklanacağı dizin.
        split="byclass",  # Veri kümesinin sınıflara göre ayrılacağını belirtir.
        train=True,  # Eğitim verisi olarak yükleneceğini belirtir.
        download=True,  # Veri kümesinin indirilmesi gerektiğini belirtir.
        transform=transforms.Compose([  # Görüntü dönüşümlerini tanımlar.
            transforms.ToTensor(),  # Görüntüyü tensöre dönüştürür.
            transforms.Normalize([0.5], [0.5])  # Görüntüyü normalize eder.
        ])
    )

    valid_labels = list(range(10, 62))   # A-Z (10-35), a-z (36-61) aralığındaki geçerli etiketleri tanımlar.
    indices = [i for i, (_, label) in enumerate(full_dataset) if label in valid_labels]  # Geçerli etiketlere sahip verilerin indekslerini bulur.
    label_map = {orig: new for new, orig in enumerate(valid_labels)}  # Orijinal etiketlerden yeni etiketlere bir eşleme oluşturur.
    subset = Subset(full_dataset, indices)  # Tam veri kümesinden geçerli etiketlere sahip bir alt küme oluşturur.

    return RemapLabels(subset, label_map), [chr(i) for i in range(65, 91)] + [chr(i) for i in range(97, 123)]  # Yeniden etiketlenmiş alt küme ve sınıf etiketlerini döndürür.


dataset, class_labels = get_letter_emnist_dataset()  # Veri kümesini ve sınıf etiketlerini alır.
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)  # Veri yükleyiciyi oluşturur.
num_classes = len(class_labels)  # Sınıf sayısını belirler.

os.makedirs("emnist_diffusion_model", exist_ok=True)  # Modelin kaydedileceği dizini oluşturur.
with open("emnist_diffusion_model/class_map.json", "w") as f:  # Sınıf etiketlerini bir JSON dosyasına kaydeder.
    json.dump(class_labels, f)  # Sınıf etiketlerini JSON formatında dosyaya yazar.

# === 2. Model Yapılandırması ===

noise_scheduler = DDPMScheduler(  # Gürültü zamanlamasını tanımlar.
    num_train_timesteps=1000,  # Eğitim için kullanılacak zaman adımlarını belirtir.
    beta_schedule="linear",  # Beta zamanlama tipini belirtir (gürültü ekleme şeması).
    beta_start=0.0001,  # Beta'nın başlangıç değerini belirtir.
    beta_end=0.02  # Beta'nın bitiş değerini belirtir.
)

model = UNet2DModel(  # UNet 2D modelini tanımlar.
    sample_size=28,  # Giriş görüntüsünün boyutunu belirtir.
    in_channels=1,  # Giriş görüntüsünün kanal sayısını belirtir (gri tonlamalı için 1).
    out_channels=1,  # Çıkış görüntüsünün kanal sayısını belirtir.
    layers_per_block=2,   # Her bloktaki katman sayısını belirtir.
    block_out_channels=(32, 64, 128),  # Her blokun çıkış kanallarını belirtir.
    down_block_types=("DownBlock2D", "DownBlock2D", "DownBlock2D"),  # Aşağı örnekleme blok tiplerini belirtir.
    up_block_types=("UpBlock2D", "UpBlock2D", "UpBlock2D"),  # Yukarı örnekleme blok tiplerini belirtir.
    class_embed_type="timestep",  # Sınıf gömme tipini belirtir.
    num_class_embeds=num_classes,  # Sınıf gömme sayısını belirtir (sınıf sayısı).
)


optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=1e-6)  # AdamW optimizasyon algoritmasını tanımlar.
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3, factor=0.5, verbose=True)  # Öğrenme oranı düşürücüyü tanımlar.

# === 3. Hızlandırıcı Kurulumu ===

accelerator = Accelerator(  # Hızlandırıcıyı tanımlar.
    mixed_precision="fp16",  # Karışık duyarlıklı eğitim kullanacağını belirtir.
    gradient_accumulation_steps=4,  # Gradyan birikim adımlarını belirtir.
    log_with="tensorboard",  # TensorBoard ile loglama yapacağını belirtir.
    project_dir="emnist_diffusion_model"  # TensorBoard loglarının saklanacağı dizini belirtir.
)

model, optimizer, dataloader = accelerator.prepare(model, optimizer, dataloader)  # Modeli, optimize ediciyi ve veri yükleyiciyi hızlandırıcı için hazırlar.

# === 4. Kontrol Noktası Yükleme ===

checkpoint_path = "emnist_diffusion_model/epoch_47"  # Yüklenecek kontrol noktasının yolunu belirtir.
start_epoch = 0  # Başlangıç epoch'unu tanımlar.
if os.path.exists(checkpoint_path):  # Eğer belirtilen yolda bir kontrol noktası varsa:
    accelerator.load_state(checkpoint_path)  # Kontrol noktasını yükler.
    start_epoch = 47  # Başlangıç epoch'unu günceller.
    print(f"Checkpoint from epoch 47 successfully loaded!")  # Başarı mesajını yazdırır.
else:  # Eğer kontrol noktası bulunamazsa:
    print(f"Checkpoint from epoch 47 not found, starting from scratch.")  # Bilgi mesajını yazdırır.

ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(accelerator.device)  # SSIM metriğini tanımlar ve hızlandırıcı cihazına taşır.
fid_metric = FrechetInceptionDistance(feature=64).to(accelerator.device)  # FID metriğini tanımlar ve hızlandırıcı cihazına taşır.
is_metric = InceptionScore().to(accelerator.device)  # IS metriğini tanımlar ve hızlandırıcı cihazına taşır.



def estimate_neurons(model):  # Modeldeki toplam nöron sayısını tahmin eden bir fonksiyon tanımlar.
    total_neurons = 0  # Toplam nöron sayısını saklamak için bir değişkeni başlatır.
    for name, module in model.named_modules():  # Modelin katmanları arasında döngü yapar.
        if isinstance(module, torch.nn.Conv2d):  # Eğer katman bir Convolutional katmansa:
            total_neurons += module.out_channels * module.kernel_size[0] * module.kernel_size[1]  # Nöron sayısını hesaplar ve toplama ekler.
        elif isinstance(module, torch.nn.Linear):  # Eğer katman bir Fully Connected katmansa:
            total_neurons += module.out_features  # Nöron sayısını toplama ekler.
        elif isinstance(module, torch.nn.Embedding):  # Eğer katman bir Embedding katmansa:
            total_neurons += module.embedding_dim  # Nöron sayısını toplama ekler.
        elif isinstance(module, torch.nn.GroupNorm):
            total_neurons += module.num_channels * 2

    return total_neurons  # Toplam nöron sayısını döndürür.

num_neurons = estimate_neurons(accelerator.unwrap_model(model))  # Modeldeki toplam nöron sayısını hesaplar.
print(f"Estimated total number of neurons in the model: {num_neurons:,}")  # Sonucu yazdırır.


# === 5. Örnekleme ve Değerlendirme ===

def generate_batch_samples(model, scheduler, num_classes, device):  # Belirli bir batch için örnekler üreten bir fonksiyon tanımlar.
    model.eval()  # Modeli değerlendirme moduna geçirir.
    # Her sınıf için bir örnek oluşturur, sıralı olarak (0'dan num_classes-1'e kadar)
    all_labels = torch.arange(num_classes, device=device)
    noise = torch.randn((num_classes, 1, 28, 28), device=device)

    for t in tqdm(reversed(range(scheduler.num_train_timesteps)), desc="Sampling"):
        t_batch = torch.full((num_classes,), t, device=device)
        with torch.no_grad():
            noise_pred = model(noise, t_batch, all_labels).sample
        noise = scheduler.step(noise_pred, t, noise).prev_sample

    images = (noise.clamp(-1, 1) + 1) * 0.5
    return images, all_labels

def get_label_ordered_real_images(dataloader, num_classes):
    """Collects one example of a real image for each class, ordered by label (from 0 to num_classes-1)"""
    ordered = {}
    for images, labels in dataloader:
        for img, label in zip(images, labels):
            label = label.item()
            if label not in ordered:
                ordered[label] = img
            if len(ordered) == num_classes:
                break
        if len(ordered) == num_classes:
            break

    # Ensure there is exactly one example for each class, in order
    ordered_images = [ordered[i] for i in range(num_classes)]
    return torch.stack(ordered_images)




def show_label_aligned_images(fake_images, fake_labels, real_images, class_labels, epoch_tag="pre_training"):  # Etiket hizalı görüntüleri gösteren bir fonksiyon tanımlar.
    num_classes = len(class_labels)

    fig, axes = plt.subplots(2, num_classes, figsize=(num_classes, 2))
    for i in range(num_classes):
        # Both fake and real images are already ordered (from 0 to num_classes-1)
        # Show the real image
        axes[0, i].imshow(real_images[i][0].cpu(), cmap="gray")
        axes[0, i].axis("off")
        axes[0, i].set_title(class_labels[i], fontsize=6)

        # Show the fake image (same index because they are ordered)
        axes[1, i].imshow(fake_images[i][0].cpu(), cmap="gray")
        axes[1, i].axis("off")

    plt.suptitle(f"Real (Top) vs Fake (Bottom) - {epoch_tag}", y=1.05)
    plt.tight_layout()
    plt.show()

def save_individual_samples(fake_images, fake_labels, class_labels, epoch):  # Üretilen örnekleri ayrı ayrı kaydeden bir fonksiyon tanımlar.
    os.makedirs(f"emnist_diffusion_model/samples/epoch_{epoch}", exist_ok=True)

    for img, label in zip(fake_images, fake_labels):
        label = label.item()
        char = class_labels[label]
        case = 'u' if char.isupper() else 'l'
        filename = f"emnist_diffusion_model/samples/epoch_{epoch}/{epoch}_{char}_{case}.png"

        img_pil = transforms.ToPILImage()(img.cpu())
        img_pil.save(filename)

def evaluate_metrics(fake_images, fake_labels, real_images, ssim_metric, fid_metric, is_metric, device):  # Üretilen görüntülerin kalitesini değerlendiren bir fonksiyon tanımlar.
    # Metrikler için gerçek ve sahte görüntüleri hazırlar
    real_images = (real_images + 1) / 2.0
    if real_images.ndim == 3:
        real_images = real_images[:, None]
    if fake_images.ndim == 3:
        fake_images = fake_images[:, None]

    # SSIM hesaplaması
    ssim_score = ssim_metric(fake_images.to(device), real_images.to(device)).item()

    # FID ve IS için hazırla
    fake_images_metric = preprocess_for_metrics(fake_images)
    real_images_metric = preprocess_for_metrics(real_images)

    # Metrikleri sıfırla
    fid_metric.reset()
    is_metric.reset()

    # FID'yi hesapla
    fid_metric.update(real_images_metric.to(device), real=True)
    fid_metric.update(fake_images_metric.to(device), real=False)
    fid_score = fid_metric.compute().item()

    # IS'yi hesapla
    is_metric.update(fake_images_metric.to(device))
    is_score, _ = is_metric.compute()
    is_score = is_score.item()

    return ssim_score, fid_score, is_score

def preprocess_for_metrics(imgs):  # Görüntüleri metrik hesaplamaları için önceden işleyen bir fonksiyon tanımlar.
    # [0, 255] aralığına ölçekle, uint8'e dönüştür
    imgs = ((imgs + 1) * 127.5).clamp(0, 255).to(torch.uint8)

    # 1 kanallıdan 3 kanallıya dönüştür
    if imgs.shape[1] == 1:
        imgs = imgs.repeat(1, 3, 1, 1)

    # 299x299'a yeniden boyutlandır
    imgs = F.interpolate(imgs.float(), size=(299, 299), mode="bilinear", align_corners=False)

    return imgs.to(torch.uint8)



history = {
    "epoch": [],
    "loss": [],
    "ssim": [],
    "fid": [],
    "is": []
}


# === 6. Eğitim Döngüsü ===

num_epochs = 100  # Toplam epoch sayısını tanımlar.
save_interval = 5  # Modeli ne sıklıkla kaydedeceğimizi tanımlar.
grad_accum_steps = accelerator.gradient_accumulation_steps  # Hızlandırıcıdan gradyan birikim adımı sayısını alır.

for epoch in range(start_epoch, num_epochs):  # Her epoch için döngü.
    model.train()  # Modeli eğitim moduna ayarlar.
    total_loss = 0.0  # Epoch için toplam kaybı başlatır.
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}")  # Epoch için bir ilerleme çubuğu oluşturur.

    for batch_idx, batch in enumerate(progress_bar):  # Veri yükleyiciden batch'ler arasında döngü yapar.
        if batch_idx % 100 == 0 and torch.cuda.is_available():  # Her 100 batch'te bir, eğer CUDA varsa:
            torch.cuda.empty_cache()  # Bellek sorunlarını önlemek için CUDA önbelleğini temizler.

        clean_images, labels = batch  # Batch'ten temiz görüntüleri ve etiketleri alır.
        clean_images = clean_images.to(accelerator.device)  # Görüntüleri hızlandırıcı cihazına (GPU/TPU) taşır.
        labels = labels.to(accelerator.device)  # Etiketleri hızlandırıcı cihazına taşır.

        noise = torch.randn_like(clean_images)  # Görüntülerle aynı boyutta rastgele gürültü oluşturur.
        timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (clean_images.size(0),), device=clean_images.device)  # Her görüntü için rastgele zaman adımları oluşturur.
        noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)  # Gürültülü görüntüleri oluşturur.
        noise_pred = model(noisy_images, timesteps, labels).sample  # Modeli gürültülü görüntülerle çalıştırarak gürültü tahminini alır.
        loss = torch.nn.functional.mse_loss(noise_pred, noise)  # Tahmin edilen gürültü ile gerçek gürültü arasındaki ortalama kare hatasını hesaplar.
        total_loss += loss.item()  # Epoch'un toplam kaybına mevcut batch'in kaybını ekler.

        accelerator.backward(loss)  # Gradyanları hesaplamak için geri yayılımı gerçekleştirir.
        if (batch_idx + 1) % grad_accum_steps == 0:  # Eğer işlenen batch sayısı gradyan birikim adımlarına eşitse:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)  # Gradyanların çok büyümesini önlemek için kırpar.
            optimizer.step()  # Modeli güncellemek için bir optimizasyon adımı gerçekleştirir.
            optimizer.zero_grad()  # Gradyanları sıfırlar.

        avg_loss = total_loss / (batch_idx + 1)  # Mevcut batch'e kadar olan ortalama kaybı hesaplar.
        progress_bar.set_postfix(loss=loss.item(), avg_loss=avg_loss)  # İlerleme çubuğunu mevcut kayıp ve ortalama kayıpla günceller.

    # Epoch sonunda değerlendirme
    model_eval = accelerator.unwrap_model(model)  # Modeli hızlandırıcıdan çıkarır.

    # Her epoch'ta örnek oluştur
    fake_images, fake_labels = generate_batch_samples(model_eval, noise_scheduler, num_classes, accelerator.device)  # Örnek görüntüler oluşturur.
    real_images = get_label_ordered_real_images(dataloader, num_classes)

    # Oluşturulan örnekleri kaydet
    save_individual_samples(fake_images, fake_labels, class_labels, epoch + 1)

    # Oluşturulan örnekleri gerçek görüntülerle karşılaştır
    print(f"\n[Epoch {epoch + 1}] Real vs Generated Samples:")
    show_label_aligned_images(fake_images, fake_labels, real_images, class_labels, epoch_tag=f"epoch_{epoch + 1}")

    # Metrikleri değerlendir
    ssim_score, fid_score, is_score = evaluate_metrics(
        fake_images, fake_labels, real_images,
        ssim_metric, fid_metric, is_metric,
        accelerator.device
    )

    # Geçmişi güncelle
    history["epoch"].append(epoch + 1)
    history["loss"].append(avg_loss)
    history["ssim"].append(ssim_score)
    history["fid"].append(fid_score)
    history["is"].append(is_score)

    print(f"[Epoch {epoch + 1}] SSIM: {ssim_score:.4f}, FID: {fid_score:.4f}, IS: {is_score:.4f}")

    # Metrikleri CSV dosyasına kaydet
    pd.DataFrame(history).to_csv("emnist_diffusion_model/training_metrics.csv", index=False)

    scheduler.step(ssim_score)  # Öğrenme oranını SSIM puanına göre ayarlar.

    # İlerlemeyi çiz
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 3, 1)
    plt.plot(history["epoch"], history["loss"], label="Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid()
    plt.title("Training Loss")

    plt.subplot(1, 3, 2)
    plt.plot(history["epoch"], history["ssim"], label="SSIM", color="green")
    plt.xlabel("Epoch")
    plt.ylabel("SSIM")
    plt.grid()
    plt.title("SSIM vs Epoch")

    plt.subplot(1, 3, 3)
    plt.plot(history["epoch"], history["fid"], label="FID", color="blue")
    plt.plot(history["epoch"], history["is"], label="IS", color="orange")
    plt.xlabel("Epoch")
    plt.ylabel("Score")
    plt.grid()
    plt.legend()
    plt.title("FID & IS vs Epoch")

    plt.tight_layout()
    plt.savefig(f"emnist_diffusion_model/training_progress_epoch_{epoch + 1}.png")
    plt.close()

    if len(history["ssim"]) >= 2 and history["ssim"][-1] < history["ssim"][-2]:
        print("⚠️ Warning: SSIM has decreased — possible overfitting.")

    accelerator.save_state(output_dir=f"emnist_diffusion_model/epoch_{epoch + 1}")  # Modeli kaydeder.

accelerator.save_state(output_dir="emnist_diffusion_model/final")  # Son modeli kaydeder.
accelerator.end_training()  # Hızlandırıcıyı sonlandırır.

print("Training completed successfully!")  # Eğitim tamamlandığında bir mesaj yazdırır.
